In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import re
teamsURL='https://www.espn.com/soccer/teams/_/league/FIFA.WORLD/fifa-world-cup'
playersURL='https://www.fifaindex.com/players/fifa18/?name='

In [2]:
def scrape_teams(teamsURL):
    teamsList=[]
    links=[]
    reqURL = requests.get(teamsURL)
    soup = BeautifulSoup(reqURL.content, "html.parser")
    for team in soup.findAll("a", class_="AnchorLink"):
        
        for teamName in team.findAll("h2", class_="di clr-gray-01 h5"):      
            teamsList.append(teamName.text)

        if team.text=='Squad':
            link=re.findall(r'href="([^"]*/)', str(team))      
            links.append("https://www.espn.com"+link[0]+'league/FIFA.WORLD')
    
    return list(zip(teamsList,links))

In [3]:
def scrape_squads(teamURLlist):
    squadDict={}
  
    for i,x in enumerate(teamURLlist):
        
        urlTeam=x[1]
        reqURL = requests.get(urlTeam)  
        soup = BeautifulSoup(reqURL.content, "html.parser")
        players=[]

    # store players from team into a squad 
        for player in soup.findAll(style="min-width:140px"):
            players.append(re.sub(r'[0-9]+', '', player.text))
            squadDict[teamURLlist[i][0]]=players

    return squadDict   


squad_dict=scrape_squads(scrape_teams(teamsURL))

In [67]:
print(squad_dict['Germany'])

[('Name', 'https://www.fifaindex.com/player/204383/barry-mcnamee/fifa18/'), ('Manuel Neuer', 'https://www.fifaindex.com/player/167495/manuel-neuer/fifa18/'), ('Kevin Trapp', 'https://www.fifaindex.com/player/188943/kevin-trapp/fifa18/'), ('Marc-André ter Stegen', 'https://www.fifaindex.com/player/192448/marc-andr%C3%A9-ter-stegen/fifa18/'), ('Marvin Plattenhardt', 'https://www.fifaindex.com/player/192557/marvin-plattenhardt/fifa18/'), ('Mats Hummels', 'https://www.fifaindex.com/player/178603/mats-hummels/fifa18/'), ('Niklas Süle', 'https://www.fifaindex.com/player/212190/niklas-s%C3%BCle/fifa18/'), ('Antonio Rüdiger', 'https://www.fifaindex.com/player/205452/antonio-r%C3%BCdiger/fifa18/'), ('Jérôme Boateng', 'https://www.fifaindex.com/player/183907/j%C3%A9r%C3%B4me-boateng/fifa18/'), ('Jonas Hector', 'https://www.fifaindex.com/player/208334/jonas-hector/fifa18/'), ('Matthias Ginter', 'https://www.fifaindex.com/player/207862/matthias-ginter/fifa18/'), ('Sami Khedira', 'https://www.fifai

In [5]:
def scrape_player_links(playerVals):  
    
    playerName=[]
    playerLink=[]
    for players in playerVals:
        try:           
            req = requests.get(playersURL+players+'&order=desc')
            soup = BeautifulSoup(req.content, "html.parser")
            for player in soup.find(class_="player"):
                    link=re.findall(r'href="([^"]*/)', str(player))
                    if players not in playerName:
                        playerName.append(players)
                        playerLink.append('https://www.fifaindex.com'+str(link[0]))
        except:
            link="No link" 
            playerName.append(players)
            playerLink.append(link)          
            pass
    return (list(zip(playerName,playerLink)))

In [6]:
for keys,values in squad_dict.items():
    squad_dict[keys]=scrape_player_links(values)

In [7]:
squad_dict['Germany']

[('Name', 'https://www.fifaindex.com/player/204383/barry-mcnamee/fifa18/'),
 ('Manuel Neuer',
  'https://www.fifaindex.com/player/167495/manuel-neuer/fifa18/'),
 ('Kevin Trapp',
  'https://www.fifaindex.com/player/188943/kevin-trapp/fifa18/'),
 ('Marc-André ter Stegen',
  'https://www.fifaindex.com/player/192448/marc-andr%C3%A9-ter-stegen/fifa18/'),
 ('Marvin Plattenhardt',
  'https://www.fifaindex.com/player/192557/marvin-plattenhardt/fifa18/'),
 ('Mats Hummels',
  'https://www.fifaindex.com/player/178603/mats-hummels/fifa18/'),
 ('Niklas Süle',
  'https://www.fifaindex.com/player/212190/niklas-s%C3%BCle/fifa18/'),
 ('Antonio Rüdiger',
  'https://www.fifaindex.com/player/205452/antonio-r%C3%BCdiger/fifa18/'),
 ('Jérôme Boateng',
  'https://www.fifaindex.com/player/183907/j%C3%A9r%C3%B4me-boateng/fifa18/'),
 ('Jonas Hector',
  'https://www.fifaindex.com/player/208334/jonas-hector/fifa18/'),
 ('Matthias Ginter',
  'https://www.fifaindex.com/player/207862/matthias-ginter/fifa18/'),
 ('Sa

In [8]:
statistics=['Country','Height','Weight','Preferred Foot','Birth Date','Age','Preferred Positions','Value','Value','Value','Wage','Wage','Wage','Joined Club','Ball Control','Dribbling','Marking','Slide Tackle','Stand Tackle','Aggression','Reactions','Att. Position','Interceptions','Vision','Composure','Crossing','Short Pass','Long Pass','Acceleration','Stamina','Strength','Balance','Sprint Speed','Agility','Jumping','Heading','Shot Power','Finishing','Long Shots','Curve','FK Acc.','Penalties','Volleys','GK Positioning','GK Diving','GK Handling','GK Kicking','GK Reflexes']

def player_stats(squad_dict):
    statsDict={}
    playerStat=[]
    for keys in squad_dict.keys():
        for val in squad_dict[keys]:
            if val[1] != "No link":
                req = requests.get(val[1])
                soup = BeautifulSoup(req.content, "html.parser")
                player_stat=soup.find(class_="col-lg-8")
                playerStat.append(keys)        
                for player in player_stat.find_all('p'):
                    try:
                        if str(player.contents[0]).strip() in statistics:
                            playerStat.append(player.contents[1].text)
                    except:
                        pass
            statsDict[val[0]]=pd.Series(playerStat)
            playerStat=[]

    return statsDict
      
playerSkills=player_stats(squad_dict)

In [18]:
df = pd.DataFrame(playerSkills).transpose()
df.columns=statistics
df.head()

,Country,Height,Weight,Preferred Foot,Birth Date,Age,Preferred Positions,Value,Value,Value,...,Long Shots,Curve,FK Acc.,Penalties,Volleys,GK Positioning,GK Diving,GK Handling,GK Kicking,GK Reflexes
Name,Uruguay,"181 cm5'11""",78 kg172 lbs,Left,"Feb. 17, 1992",26,CAMRM,€575.000,$625.000,£500.000,...,58,57,58,41,55,12,7,11,7,14
Nahuel Guzmán,Argentina,"193 cm6'4""",90 kg198 lbs,Right,"Feb. 10, 1986",32,GK,€6.500.000,$7.000.000,£5.500.000,...,17,18,22,12,12,79,77,76,67,80
Franco Armani,Argentina,"189 cm6'2""",88 kg194 lbs,Right,"Oct. 16, 1986",31,GK,€6.500.000,$7.000.000,£5.500.000,...,17,14,16,20,15,78,78,73,69,81
Wilfredo Caballero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gabriel Mercado,Argentina,"180 cm5'11""",83 kg183 lbs,Right,"March 18, 1987",31,CBRB,€9.500.000,$11.000.000,£8.500.000,...,59,68,54,49,36,12,12,16,10,6


In [19]:
df.columns

Index(['Country', 'Height', 'Weight', 'Preferred Foot', 'Birth Date', 'Age',
       'Preferred Positions', 'Value', 'Value', 'Value', 'Wage', 'Wage',
       'Wage', 'Joined Club', 'Ball Control', 'Dribbling', 'Marking',
       'Slide Tackle', 'Stand Tackle', 'Aggression', 'Reactions',
       'Att. Position', 'Interceptions', 'Vision', 'Composure', 'Crossing',
       'Short Pass', 'Long Pass', 'Acceleration', 'Stamina', 'Strength',
       'Balance', 'Sprint Speed', 'Agility', 'Jumping', 'Heading',
       'Shot Power', 'Finishing', 'Long Shots', 'Curve', 'FK Acc.',
       'Penalties', 'Volleys', 'GK Positioning', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Reflexes'],
      dtype='object')

In [20]:
df.iloc[0]

Country                      Uruguay
Height                   181 cm5'11"
Weight                  78 kg172 lbs
Preferred Foot                  Left
Birth Date             Feb. 17, 1992
Age                               26
Preferred Positions            CAMRM
Value                       €575.000
Value                       $625.000
Value                       £500.000
Wage                          €1.500
Wage                          $1.700
Wage                          £1.300
Joined Club             Dec. 1, 2017
Ball Control                      64
Dribbling                         65
Marking                           38
Slide Tackle                      47
Stand Tackle                      52
Aggression                        60
Reactions                         55
Att. Position                     57
Interceptions                     50
Vision                            65
Composure                         61
Crossing                          57
Short Pass                        64
L

In [21]:
df=df.dropna() #1

df.head()
df["Height"]=df["Height"].str.extract(r'(^\d*)')#2
df["Weight"]=df["Weight"].str.extract(r'(^\d*)')

positions=['GK','SW','RWB','RB','CB','LB','LWB','CDM','RM','CM','LM','CAM','CF','RW','ST','LW']#3
for x,column in enumerate(df['Preferred Positions']):
  temp=[]
  for i in positions:
      if i in column:
       temp.append(i)
       df.loc[df.index[x],'Preferred Positions']="/".join(temp)

df.loc[:,"Ball Control":"GK Reflexes"]=df.loc[:,"Ball Control":"GK Reflexes"].astype(int)#4

In [22]:
df

,Country,Height,Weight,Preferred Foot,Birth Date,Age,Preferred Positions,Value,Value,Value,...,Long Shots,Curve,FK Acc.,Penalties,Volleys,GK Positioning,GK Diving,GK Handling,GK Kicking,GK Reflexes
Name,Uruguay,181,78,Left,"Feb. 17, 1992",26,RM/CAM,€575.000,$625.000,£500.000,...,58,57,58,41,55,12,7,11,7,14
Nahuel Guzmán,Argentina,193,90,Right,"Feb. 10, 1986",32,GK,€6.500.000,$7.000.000,£5.500.000,...,17,18,22,12,12,79,77,76,67,80
Franco Armani,Argentina,189,88,Right,"Oct. 16, 1986",31,GK,€6.500.000,$7.000.000,£5.500.000,...,17,14,16,20,15,78,78,73,69,81
Gabriel Mercado,Argentina,180,83,Right,"March 18, 1987",31,RB/CB,€9.500.000,$11.000.000,£8.500.000,...,59,68,54,49,36,12,12,16,10,6
Nicolás Tagliafico,Argentina,172,67,Left,"Aug. 31, 1992",26,CB/LB,€13.000.000,$14.500.000,£11.500.000,...,44,61,40,46,38,15,13,9,10,16
Cristian Ansaldi,Argentina,181,76,Left,"Sept. 20, 1986",31,LB/LM,€9.000.000,$10.000.000,£8.000.000,...,66,77,45,52,51,8,15,15,11,8
Federico Fazio,Argentina,195,85,Right,"March 17, 1987",31,CB,€19.000.000,$21.000.000,£16.500.000,...,26,46,37,33,35,11,8,9,12,14
Marcos Rojo,Argentina,187,80,Left,"March 20, 1990",28,CB,€14.000.000,$16.000.000,£12.500.000,...,63,67,57,64,49,15,7,8,12,10
Nicolás Otamendi,Argentina,183,81,Right,"Feb. 12, 1988",30,CB,€30.000.000,$33.500.000,£26.500.000,...,56,50,39,45,57,11,12,5,8,12
Cristian Pavón,Argentina,174,76,Right,"Jan. 21, 1996",22,RM/RW,€12.000.000,$13.500.000,£10.500.000,...,83,72,77,59,57,13,11,10,9,7


In [43]:
bestEleven = pd.DataFrame()

In [45]:
# Goalkeeper
df_GK = df[df['Preferred Positions']=='GK']
df_GK['Tot'] = df_GK[['GK Reflexes', 'GK Diving', 'GK Handling', 'GK Positioning', 'GK Kicking','Strength','Jumping','Reactions','Balance','Composure']].sum(axis=1)
bestEleven = bestEleven.append(df_GK.sort_values(by='Tot', ascending=False).head(1))
df_GK.sort_values(by='Tot', ascending=False).head(5)

C:\Users\anike\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country,Height,Weight,Preferred Foot,Birth Date,Age,Preferred Positions,Value,Value,Value,...,Curve,FK Acc.,Penalties,Volleys,GK Positioning,GK Diving,GK Handling,GK Kicking,GK Reflexes,Tot
Manuel Neuer,Germany,193,92,Right,"March 27, 1986",32,GK,€61.000.000,$68.000.000,£53.500.000,...,14,11,47,11,91,91,90,95,89,806.0
Marc-André ter Stegen,Germany,187,85,Right,"April 30, 1992",26,GK,€44.500.000,$49.500.000,£39.000.000,...,18,12,25,14,84,86,84,87,89,777.0
David de Gea,Spain,193,76,Right,"Nov. 7, 1990",27,GK,€74.500.000,$83.500.000,£65.500.000,...,21,19,40,13,88,90,85,87,93,771.0
Anthony Lopes,Portugal,184,81,Left,"Oct. 1, 1990",27,GK,€26.000.000,$29.500.000,£23.000.000,...,15,14,16,13,81,86,80,74,87,757.0
Pepe Reina,Spain,188,92,Right,"Aug. 31, 1982",36,GK,€10.500.000,$11.500.000,£9.000.000,...,11,11,24,11,82,80,83,82,82,756.0


In [46]:
# LB
df_LB=df[df['Preferred Positions'].str.contains('LB')]
df_LB['Tot'] = df_LB[['Strength', 'Jumping', 'Stand Tackle', 'Aggression', 'Slide Tackle','Heading','Marking','Interceptions','Stamina','Reactions']].sum(axis=1)
bestEleven = bestEleven.append(df_LB.sort_values(by='Tot', ascending=False).head(1))
df_LB.sort_values(by='Tot', ascending=False).head(5)

C:\Users\anike\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country,Height,Weight,Preferred Foot,Birth Date,Age,Preferred Positions,Value,Value,Value,...,Curve,FK Acc.,Penalties,Volleys,GK Positioning,GK Diving,GK Handling,GK Kicking,GK Reflexes,Tot
Marcelo,Brazil,174,80,Left,"May 12, 1988",30,LB,€38.000.000,$42.500.000,£33.500.000,...,80,67,59,54,5,12,5,5,9,813.0
Aleksandar Kolarov,Serbia,187,83,Left,"Nov. 10, 1985",32,CB/LB,€11.500.000,$12.500.000,£10.000.000,...,85,86,74,64,16,16,15,8,6,813.0
Filipe Luís,Brazil,182,73,Left,"Aug. 9, 1985",33,LB,€29.000.000,$32.500.000,£25.500.000,...,74,75,65,56,9,11,15,9,5,809.0
Jordi Alba,Spain,170,68,Left,"March 21, 1989",29,LB,€40.000.000,$44.500.000,£35.000.000,...,77,64,59,60,6,13,15,13,13,793.0
Nacho,Spain,180,76,Right,"Jan. 18, 1990",28,RB/CB/LB,€17.500.000,$19.500.000,£15.500.000,...,55,37,45,38,15,12,7,10,7,791.0


In [51]:
#CB's
df_CB=df[df['Preferred Positions'].str.contains('CB')]
df_CB['Tot'] = df_CB[['Acceleration', 'Heading', 'Composure', 'Balance', 'Agility','Jumping','Stand Tackle','Slide Tackle','Aggression','Strength']].sum(axis=1)
bestEleven = bestEleven.append(df_CB.sort_values(by='Tot', ascending=False).head(2))
df_CB.sort_values(by='Tot', ascending=False).head(5)

C:\Users\anike\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country,Height,Weight,Preferred Foot,Birth Date,Age,Preferred Positions,Value,Value,Value,...,Curve,FK Acc.,Penalties,Volleys,GK Positioning,GK Diving,GK Handling,GK Kicking,GK Reflexes,Tot
Sergio Ramos,Spain,184,82,Right,"March 30, 1986",32,CB,€52.000.000,$58.500.000,£46.000.000,...,73,67,68,66,7,11,8,9,11,824.0
Thiago Silva,Brazil,183,82,Right,"Sept. 22, 1984",33,CB,€29.000.000,$32.500.000,£25.500.000,...,61,67,69,63,9,9,12,5,10,797.0
Nicolás Otamendi,Argentina,183,81,Right,"Feb. 12, 1988",30,CB,€30.000.000,$33.500.000,£26.500.000,...,50,39,45,57,11,12,5,8,12,794.0
Diego Godín,Uruguay,187,79,Right,"Feb. 16, 1986",32,CB,€40.000.000,$44.500.000,£35.000.000,...,49,51,50,47,5,6,8,15,15,791.0
César Azpilicueta,Spain,178,78,Right,"Aug. 28, 1989",29,CB,€43.500.000,$49.000.000,£38.500.000,...,68,58,66,51,6,13,9,10,5,787.0


In [48]:
# RB
df_RB=df[df['Preferred Positions'].str.contains('RB')]
df_RB['Tot'] = df_RB[['Acceleration', 'Sprint Speed', 'Stamina', 'Balance', 'Agility','Jumping','Stand Tackle','Slide Tackle','Aggression','Strength']].sum(axis=1)
bestEleven = bestEleven.append(df_RB.sort_values(by='Tot', ascending=False).head(1))
df_RB.sort_values(by='Tot', ascending=False).head(5)

C:\Users\anike\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country,Height,Weight,Preferred Foot,Birth Date,Age,Preferred Positions,Value,Value,Value,...,Curve,FK Acc.,Penalties,Volleys,GK Positioning,GK Diving,GK Handling,GK Kicking,GK Reflexes,Tot
Kyle Walker,England,183,70,Right,"May 28, 1990",28,RWB/RB/RW,€25.500.000,$28.500.000,£22.500.000,...,76,71,63,57,15,12,6,16,8,821.0
Igor Smolnikov,Russia,178,70,Right,"Aug. 8, 1988",30,RB,€10.000.000,$11.500.000,£9.000.000,...,49,46,41,57,8,7,12,10,15,798.0
Santiago Arias,Colombia,175,72,Right,"Jan. 13, 1992",26,RB,€10.500.000,$11.500.000,£9.000.000,...,31,41,56,43,15,14,7,14,16,794.0
Ricardo Pereira,Portugal,175,70,Right,"Oct. 6, 1993",24,RB/RM,€20.000.000,$22.500.000,£17.500.000,...,76,42,56,53,11,16,12,15,11,793.0
Jefferson Lerma,Colombia,179,70,Right,"Oct. 25, 1994",23,RB/CDM/CM,€10.000.000,$11.500.000,£9.000.000,...,67,68,58,53,7,16,14,11,9,791.0


In [53]:
# 2 CMs
df_CM=df[df['Preferred Positions'].str.contains('CM')]
df_CM['Tot'] = df_CM[['Acceleration', 'Sprint Speed', 'Dribbling', 'Balance', 'Agility','Ball Control','Long Pass','Short Pass','Marking','Vision']].sum(axis=1)
bestEleven = bestEleven.append(df_CM.sort_values(by='Tot', ascending=False).head(2))
df_CM.sort_values(by='Tot', ascending=False).head(5)

C:\Users\anike\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country,Height,Weight,Preferred Foot,Birth Date,Age,Preferred Positions,Value,Value,Value,...,Curve,FK Acc.,Penalties,Volleys,GK Positioning,GK Diving,GK Handling,GK Kicking,GK Reflexes,Tot
Luka Modric,Croatia,172,66,Right,"Sept. 9, 1985",33,CDM/CM,€57.000.000,$63.500.000,£50.000.000,...,79,77,80,74,14,13,9,7,9,843.0
Thiago,Spain,174,70,Right,"April 11, 1991",27,CDM/CM/CAM,€68.500.000,$77.000.000,£60.500.000,...,85,77,75,90,9,6,11,7,13,820.0
Andrés Iniesta,Spain,171,68,Right,"May 11, 1984",34,CM/LM,€29.500.000,$33.000.000,£26.000.000,...,80,70,71,74,13,6,13,6,7,814.0
N'Golo Kanté,France,168,68,Right,"March 29, 1991",27,CDM/CM,€59.500.000,$67.000.000,£52.500.000,...,49,49,54,56,7,15,12,10,10,813.0
Mateo Kovacic,Croatia,178,78,Right,"May 6, 1994",24,CDM/CM,€30.000.000,$33.500.000,£26.500.000,...,78,68,59,76,7,9,5,6,7,812.0


In [56]:
#LW
df_LW=df[df['Preferred Positions'].str.contains('LW')]
df_LW['Tot'] = df_LW[['Acceleration', 'Sprint Speed', 'Dribbling', 'Balance', 'Agility','Ball Control','Shot Power','Short Pass','Crossing','Finishing']].sum(axis=1)
bestEleven = bestEleven.append(df_LW.sort_values(by='Tot', ascending=False).head(1))
df_LW.sort_values(by='Tot', ascending=False).head(5)

C:\Users\anike\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country,Height,Weight,Preferred Foot,Birth Date,Age,Preferred Positions,Value,Value,Value,...,Curve,FK Acc.,Penalties,Volleys,GK Positioning,GK Diving,GK Handling,GK Kicking,GK Reflexes,Tot
Eden Hazard,Belgium,173,76,Right,"Jan. 7, 1991",27,CF/LW,€95.500.000,$107.000.000,£84.000.000,...,83,79,86,80,8,11,12,6,8,887.0
Neymar,Brazil,175,68,Right,"Feb. 5, 1992",26,LW,€119.500.000,$133.500.000,£105.000.000,...,82,84,81,83,15,9,9,15,11,878.0
Cristiano Ronaldo,Portugal,187,83,Right,"Feb. 5, 1985",33,ST/LW,€95.500.000,$107.000.000,£84.000.000,...,81,76,85,88,14,7,11,15,11,872.0
Douglas Costa,Brazil,172,70,Left,"Sept. 14, 1990",27,LM/RW/LW,€33.500.000,$37.500.000,£29.500.000,...,82,78,77,76,12,13,15,9,5,862.0
Philippe Coutinho,Brazil,172,68,Right,"June 12, 1992",26,RM/CM/LM/LW,€63.000.000,$70.500.000,£55.500.000,...,84,84,70,75,14,12,7,9,6,850.0


In [59]:
#CAM
df_CAM=df[df['Preferred Positions'].str.contains('CAM')]
df_CAM['Tot'] = df_CAM[['Acceleration', 'Vision', 'Dribbling', 'Balance', 'Composure','Ball Control','Shot Power','Short Pass','Crossing','Finishing']].sum(axis=1)
bestEleven = bestEleven.append(df_CAM.sort_values(by='Tot', ascending=False).head(1))
df_CAM.sort_values(by='Tot', ascending=False).head(5)

C:\Users\anike\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country,Height,Weight,Preferred Foot,Birth Date,Age,Preferred Positions,Value,Value,Value,...,Curve,FK Acc.,Penalties,Volleys,GK Positioning,GK Diving,GK Handling,GK Kicking,GK Reflexes,Tot
Kevin De Bruyne,Belgium,181,68,Right,"June 28, 1991",27,CM/CAM,€104.500.000,$117.000.000,£92.000.000,...,85,83,77,82,10,15,13,5,13,866.0
Paulo Dybala,Argentina,177,75,Left,"Nov. 15, 1993",24,CAM/ST,€89.000.000,$100.000.000,£78.500.000,...,88,88,86,88,5,5,4,4,8,863.0
Christian Eriksen,Denmark,182,76,Right,"Feb. 14, 1992",26,RM/CM/CAM,€74.500.000,$83.500.000,£65.500.000,...,86,86,67,74,7,9,14,7,6,851.0
Nabil Fekir,France,173,75,Left,"July 18, 1993",25,CAM/ST,€40.500.000,$45.000.000,£35.500.000,...,81,78,77,77,14,15,9,8,15,849.0
Isco,Spain,176,79,Right,"April 21, 1992",26,CM/CAM,€55.000.000,$61.500.000,£48.500.000,...,86,74,76,65,15,10,8,12,6,845.0


In [61]:
#RW
df_RW=df[df['Preferred Positions'].str.contains('RW')]
df_RW['Tot'] = df_RW[['Sprint Speed', 'Vision', 'Dribbling', 'Balance', 'Composure','Ball Control','Shot Power','Stamina','Crossing','Finishing']].sum(axis=1)
bestEleven = bestEleven.append(df_RW.sort_values(by='Tot', ascending=False).head(1))
df_RW.sort_values(by='Tot', ascending=False).head(5)

C:\Users\anike\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country,Height,Weight,Preferred Foot,Birth Date,Age,Preferred Positions,Value,Value,Value,...,Curve,FK Acc.,Penalties,Volleys,GK Positioning,GK Diving,GK Handling,GK Kicking,GK Reflexes,Tot
Lionel Messi,Argentina,170,72,Left,"June 24, 1987",31,CF/RW/ST,€118.500.000,$133.000.000,£104.500.000,...,90,92,75,86,14,6,11,15,8,894.0
Mohamed Salah,Egypt,175,71,Left,"June 15, 1992",26,RW,€63.000.000,$70.500.000,£55.500.000,...,82,60,61,73,11,14,14,9,14,855.0
Antoine Griezmann,France,175,71,Left,"March 21, 1991",27,RW/ST/LW,€71.000.000,$79.500.000,£62.500.000,...,84,75,72,87,13,14,8,14,14,831.0
Douglas Costa,Brazil,172,70,Left,"Sept. 14, 1990",27,LM/RW/LW,€33.500.000,$37.500.000,£29.500.000,...,82,78,77,76,12,13,15,9,5,827.0
Bernardo Silva,Portugal,173,64,Left,"Aug. 10, 1994",24,RW,€42.500.000,$47.500.000,£37.500.000,...,83,63,64,66,12,9,10,14,9,822.0


In [64]:
#ST
df_ST=df[df['Preferred Positions'].str.contains('ST')]
df_ST['Tot'] = df_ST[['Sprint Speed', 'Acceleration', 'Strength', 'Balance', 'Agility','Ball Control','Shot Power','Stamina','Heading','Finishing']].sum(axis=1)
bestEleven = bestEleven.append(df_ST.sort_values(by='Tot', ascending=False).head(1))
df_ST.sort_values(by='Tot', ascending=False).head(5)

C:\Users\anike\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country,Height,Weight,Preferred Foot,Birth Date,Age,Preferred Positions,Value,Value,Value,...,Curve,FK Acc.,Penalties,Volleys,GK Positioning,GK Diving,GK Handling,GK Kicking,GK Reflexes,Tot
Cristiano Ronaldo,Portugal,187,83,Right,"Feb. 5, 1985",33,ST/LW,€95.500.000,$107.000.000,£84.000.000,...,81,76,85,88,14,7,11,15,11,873.0
Luis Suárez,Uruguay,182,86,Right,"Jan. 24, 1987",31,ST,€97.000.000,$109.000.000,£85.500.000,...,86,84,85,88,33,27,25,31,37,850.0
Sergio Agüero,Argentina,173,70,Right,"June 2, 1988",30,ST,€66.500.000,$74.500.000,£58.500.000,...,82,72,83,85,11,13,15,6,14,845.0
Lionel Messi,Argentina,170,72,Left,"June 24, 1987",31,CF/RW/ST,€118.500.000,$133.000.000,£104.500.000,...,90,92,75,86,14,6,11,15,8,843.0
Gabriel Jesus,Brazil,175,73,Right,"April 3, 1997",21,ST,€41.000.000,$46.000.000,£36.000.000,...,76,54,78,72,6,13,8,6,15,836.0


**Best 11 that is obtained by the total of top 10 features for each position:**

In [65]:
bestEleven

,Country,Height,Weight,Preferred Foot,Birth Date,Age,Preferred Positions,Value,Value,Value,...,Curve,FK Acc.,Penalties,Volleys,GK Positioning,GK Diving,GK Handling,GK Kicking,GK Reflexes,Tot
Manuel Neuer,Germany,193,92,Right,"March 27, 1986",32,GK,€61.000.000,$68.000.000,£53.500.000,...,14,11,47,11,91,91,90,95,89,806.0
Marcelo,Brazil,174,80,Left,"May 12, 1988",30,LB,€38.000.000,$42.500.000,£33.500.000,...,80,67,59,54,5,12,5,5,9,813.0
Kyle Walker,England,183,70,Right,"May 28, 1990",28,RWB/RB/RW,€25.500.000,$28.500.000,£22.500.000,...,76,71,63,57,15,12,6,16,8,821.0
Sergio Ramos,Spain,184,82,Right,"March 30, 1986",32,CB,€52.000.000,$58.500.000,£46.000.000,...,73,67,68,66,7,11,8,9,11,824.0
Thiago Silva,Brazil,183,82,Right,"Sept. 22, 1984",33,CB,€29.000.000,$32.500.000,£25.500.000,...,61,67,69,63,9,9,12,5,10,797.0
Luka Modric,Croatia,172,66,Right,"Sept. 9, 1985",33,CDM/CM,€57.000.000,$63.500.000,£50.000.000,...,79,77,80,74,14,13,9,7,9,843.0
Thiago,Spain,174,70,Right,"April 11, 1991",27,CDM/CM/CAM,€68.500.000,$77.000.000,£60.500.000,...,85,77,75,90,9,6,11,7,13,820.0
Eden Hazard,Belgium,173,76,Right,"Jan. 7, 1991",27,CF/LW,€95.500.000,$107.000.000,£84.000.000,...,83,79,86,80,8,11,12,6,8,887.0
Kevin De Bruyne,Belgium,181,68,Right,"June 28, 1991",27,CM/CAM,€104.500.000,$117.000.000,£92.000.000,...,85,83,77,82,10,15,13,5,13,866.0
Lionel Messi,Argentina,170,72,Left,"June 24, 1987",31,CF/RW/ST,€118.500.000,$133.000.000,£104.500.000,...,90,92,75,86,14,6,11,15,8,894.0
